##RUNNING PYSPARK

In [2]:

import os
import urllib.request
import ssl

data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

data_dir1 = "hadoop/bin"
os.makedirs(data_dir1, exist_ok=True)

urls_and_paths = {
    "https://raw.githubusercontent.com/Yadav-Raj-Ghimire/BigData_work/refs/heads/main/spark-config/test.txt": os.path.join(data_dir, "test.txt"),
    "https://github.com/Yadav-Raj-Ghimire/BigData_work/tree/main/spark-config/hadoop/bin/winutils.exe": os.path.join(data_dir1, "winutils.exe"),
    "https://github.com/Yadav-Raj-Ghimire/BigData_work/tree/main/spark-config/hadoop/bin/hadoop.dll": os.path.join(data_dir1, "hadoop.dll")
}


# Create an unverified SSL context
ssl_context = ssl._create_unverified_context()

for url, path in urls_and_paths.items():
    # Use the unverified context with urlopen
    with urllib.request.urlopen(url, context=ssl_context) as response, open(path, 'wb') as out_file:
        data = response.read()
        out_file.write(data)
import os, urllib.request, ssl; ssl_context = ssl._create_unverified_context(); [open(path, 'wb').write(urllib.request.urlopen(url, context=ssl_context).read())]


# ======================================================================================

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import sys

python_path = sys.executable
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['HADOOP_HOME'] ="hadoop"
os.environ['JAVA_HOME'] = r'C:\Users\yrghimire\.jdks\corretto-1.8.0_482'
######################🔴🔴🔴################################

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.5.1 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.5.4 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4 pyspark-shell'


conf = SparkConf().setAppName("pyspark").setMaster("local[*]").set("spark.driver.host","localhost").set("spark.default.parallelism", "1")
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()

spark.read.format("csv").load("data/test.txt").toDF("Success").show(20, False)


##################🔴🔴🔴🔴🔴🔴 -> DONT TOUCH ABOVE CODE -- TYPE BELOW ####################################

print()


+---------------------+
|Success              |
+---------------------+
|Spark Journey Started|
+---------------------+




In [3]:
# Reading CSV file from given local location of Laptop
train_df = spark.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").load("Dataset/TrainData1.txt")
print("\n=====train_df DF======\n")
train_df.show(5)


test_df = spark.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").load("Dataset/TestData1.txt")
print("\n=====test_df DF======\n")
#test_df.show(5)

print("\n===SCHEMA=====\n")

#train_df.printSchema()

#test_df.printSchema()




=====train_df DF======

+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----

In [4]:
print("Train rows:", train_df.count())
print("Test rows:", test_df.count())

Train rows: 150
Test rows: 53


In [5]:
train_label_df = spark.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").load("Dataset/TrainLabel1.txt")
print("\n=====train_label_df DF======\n")
train_label_df.show(5)

print("TrainLabel rows:", train_label_df.count())


=====train_label_df DF======

+---+
|_c0|
+---+
|  1|
|  1|
|  2|
|  1|
|  1|
+---+
only showing top 5 rows

TrainLabel rows: 150


In [6]:
from pyspark.sql.functions import monotonically_increasing_id

train_df = train_df.withColumn("row_id", monotonically_increasing_id())
train_label_df = train_label_df.withColumn("row_id", monotonically_increasing_id())
train_df.show(5)
train_label_df.show(5)

+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-------------

In [7]:
train_label_df = train_label_df.withColumnRenamed("_c0", "label")
train_label_df.show(5)

+-----+------+
|label|row_id|
+-----+------+
|    1|     0|
|    1|     1|
|    2|     2|
|    1|     3|
|    1|     4|
+-----+------+
only showing top 5 rows



In [8]:
train_full_df = train_df.join(train_label_df, on="row_id").drop("row_id")

train_full_df.show(5)

+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-------------

In [9]:
train_full_df.select(train_full_df.columns[-1]).distinct().show()

+-----+
|label|
+-----+
|    1|
|    3|
|    5|
|    4|
|    2|
+-----+



###Detect Missing Values

In [10]:
from pyspark.sql.functions import col

train_full_df.filter(col("_c0") == 1e99).count()


2

In [11]:
from pyspark.sql.functions import sum, when

missing_counts = train_full_df.select([
    sum(when(col(c) == 1e99, 1).otherwise(0)).alias(c)
    for c in train_full_df.columns if c != "label"
])

missing_counts.show()

+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

###Replace 1e99 with NULL

In [12]:
train_full_df = train_full_df.replace(1e99, None)


In [13]:
train_full_df.show(5)

+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-------------

In [17]:
train_full_df.createOrReplaceTempView("train_table")
train_full_df.show

<bound method DataFrame.show of DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double, _c8: double, _c9: double, _c10: double, _c11: double, _c12: double, _c13: double, _c14: double, _c15: double, _c16: double, _c17: double, _c18: double, _c19: double, _c20: double, _c21: double, _c22: double, _c23: double, _c24: double, _c25: double, _c26: double, _c27: double, _c28: double, _c29: double, _c30: double, _c31: double, _c32: double, _c33: double, _c34: double, _c35: double, _c36: double, _c37: double, _c38: double, _c39: double, _c40: double, _c41: double, _c42: double, _c43: double, _c44: double, _c45: double, _c46: double, _c47: double, _c48: double, _c49: double, _c50: double, _c51: double, _c52: double, _c53: double, _c54: double, _c55: double, _c56: double, _c57: double, _c58: double, _c59: double, _c60: double, _c61: double, _c62: double, _c63: double, _c64: double, _c65: double, _c66: double, _c67: double, _c68: double, _c